In [ ]:
# @title 🚀 全面升級數據整合平台 (v16) - 啟動器
#@markdown ---
#@markdown ### **核心架構：模組化、高穩定性、GitHub 直連**
#@markdown 歡迎使用數據整合平台 v16！此版本基於強化的模組化工程構建，提供前所未有的執行穩定性與更清晰的程式碼結構。
#@markdown - **智慧型程式碼更新**：啟動器會自動從您指定的 GitHub 儲存庫拉取最新程式碼。
#@markdown - **精準依賴管理**：透過 Poetry 精確鎖定套件版本，避免環境衝突，保障執行一致性。
#@markdown - **一站式參數配置**：所有可調整參數均集中於下方，方便您快速完成設定。
#@markdown ---
#@markdown ### ⚙️ **執行核心設定**
#@markdown ---
#@markdown #### **(A) 系統與程式碼來源**
#@markdown 指定數據平台的核心程式碼來源。除非您是開發者或使用自訂版本，否則請保留預設值。
repo_url = "https://github.com/hsp1234-web/taifexd-date.git" #@param {type:"string"}
#@markdown ---
#@markdown #### **(B) 執行模式選擇**
#@markdown ✅ **Google Drive 整合模式**：勾選此項，您的專案資料夾與產出將同步並永久保存於您的 Google Drive。
#@markdown ❌ **本地臨時模式**：取消勾選，所有檔案將儲存於此 Colab 的臨時空間，適合快速測試，但關閉分頁後資料會遺失。
enable_google_drive_integration = True #@param {type:"boolean"}
#@markdown ---
#@markdown #### **(C) 主要路徑與檔案名稱**
#@markdown 設定您在 Google Drive 中的專案根目錄，以及輸出的資料庫與日誌檔名。
project_folder = "MyTaifexDataProject" #@param {type:"string"}
database_name = "processed_data.duckdb" #@param {type:"string"}
log_name = "pipeline.log" #@param {type:"string"}
#@markdown --- 
#@markdown #### **(D) 特定檔案處理**
#@markdown 若要指定處理單一或多個ZIP檔，請在此輸入檔名，並用逗號 (`,`) 分隔。留空則代表處理輸入目錄中的全部 ZIP 檔案。
target_zip_files = "" #@param {type:"string"}
#@markdown ---
#@markdown #### **(E) 開發者選項 (Developer)**
#@markdown ⚠️ **除錯模式**：僅在需要深入分析問題時開啟。將啟用詳細日誌與硬體效能監控，日誌檔案會變得非常大。
debug_mode = False #@param {type:"boolean"}

# ======================================================================
#         穩健執行區 (請勿修改此線以下內容)
# ======================================================================
import os
import sys
import shutil
from google.colab import drive

print("--- 參數動態顯示 (執行時更新) ---")
print(f"程式碼來源: {repo_url}")
print(f"Google Drive 整合模式: {'啟用' if enable_google_drive_integration else '停用 (本地測試模式)'}")
print(f"專案資料夾: {project_folder}")
print(f"輸出資料庫: {database_name}")
print(f"輸出日誌檔: {log_name}")
print(f"指定ZIP檔: {'全部' if not target_zip_files else target_zip_files}")
print("--------------------\n")

# --- 動態路徑設定 ---
if enable_google_drive_integration:
    base_path = "/content/drive/MyDrive"
    print(f"ℹ️ Google Drive 整合已啟用。專案資料夾將嘗試建立於: {base_path}")
else:
    base_path = "/content"
    print(f"ℹ️ 本地測試模式已啟用。專案資料夾將建立於 Colab 臨時空間: {base_path}")
    print("⚠️ 注意：在本地測試模式下，所有產出檔案將在 Colab 工作階段結束後遺失。")

project_output_path = os.path.join(base_path, project_folder) 
print(f"ℹ️ 完整專案輸出路徑預期為: {project_output_path}\n")

# 步驟 1: 掛載 Google Drive (如果啟用)
if enable_google_drive_integration:
    try:
        print("⏳ 正在請求掛載 Google Drive...")
        drive.mount('/content/drive', force_remount=True)
        print("✅ Google Drive 掛載成功！")
    except Exception as e:
        print(f"❌ Google Drive 掛載失敗: {e}")
        print("\n🤔 發生了什麼事？")
        print("   看起來您可能在彈出的授權視窗中選擇了「取消」，或是網路連線出現了問題。")
        print("\n👉 您可以怎麼做？")
        print("   1. 請點擊上方儲存格左側的 ▶️ 按鈕，再次執行一次，並在彈出視窗中點擊「連線至 Google 雲端硬碟」並「允許」授權。")
        sys.exit() # 中止執行
else:
    print("ℹ️ Google Drive 整合已停用。所有操作將在本地臨時環境進行。")

try:
    # 步驟 2: 下載/更新專案程式碼
    print("\n⏳ 正在準備專案程式碼...")
    os.chdir('/content') 
    print(f"  - 目前工作目錄: {os.getcwd()}")

    repo_name = 'taifexd-date' # 根據您的專案固定
    repo_path = os.path.join('/content', repo_name)

    if os.path.exists(repo_path):
        print(f"  - 偵測到舊的 '{repo_path}' 目錄，正在刪除以確保同步最新版本...")
        shutil.rmtree(repo_path) 
    else:
        print(f"  - '{repo_path}' 目錄不存在，無需刪除。")

    print(f"  - 執行 git clone，開始從 {repo_url} 下載專案至 {repo_name}...")
    get_ipython().system(f"git clone -q {repo_url} {repo_name}")

    # 檢查 clone 是否成功
    if not os.path.exists(repo_path):
        print("❌ Git Clone 失敗!")
        print("🤔 發生了什麼事？")
        print("   看起來專案程式碼沒有成功下載到 Colab 環境中。這通常是因為：")
        print("   1. GitHub 儲存庫網址錯誤或不再存在。")
        print("   2. 儲存庫是私有的，而 Colab 沒有存取權限。")
        print("👉 您可以怎麼做？")
        print(f"   1. 請檢查您在上方 (A) 區塊設定的 `repo_url` (目前是：{repo_url}) 是否正確且可公開存取。")
        print("   2. 如果是私有儲存庫，請確保您已採取適當的授權措施 (例如，改用包含 Personal Access Token 的 URL)。")
        sys.exit() # 中止執行

    print(f"  - 切換工作目錄至 {repo_path}...")
    os.chdir(repo_path) 
    print(f"✅ 專案程式碼準備完成！目前工作目錄: {os.getcwd()}\n")

    # 步驟 3: 同步鎖定檔並安裝依賴
    print("\n⏳ 正在同步 lock 檔案並安裝依賴 (此過程可能需要數分鐘，請稍候)...")
    get_ipython().system("pip install poetry -q")
    print("  - 執行 poetry lock 來產生與 pyproject.toml 同步的鎖定檔案...")
    get_ipython().system("poetry lock")
    print("  - 執行 poetry install 來安裝所有必要的套件...")
    get_ipython().system("poetry install --no-root --without dev")
    print("✅ 所有精確版本的 Python 套件均已安裝完成！\n")

    # 步驟 4: 建構並執行主程式
    print("\n🚀 即將啟動數據整合平台主程式...")
    command_parts = [
        "poetry run python main.py",
        f"--project-folder-name='{project_folder}'",
        f"--database-name='{database_name}'",
        f"--log-name='{log_name}'",
        f"--zip-files='{target_zip_files}'"
    ]
    if not enable_google_drive_integration:
        command_parts.append("--no-gdrive")
    
    if debug_mode:
        command_parts.append("--debug")

    command = " ".join(command_parts)

    print(f"  - 將執行的最終指令:")
    print(f"    {command}")
    print("-" * 20 + " 程式開始執行 " + "-" * 20)

    get_ipython().system(command)

    print("-" * 20 + " 程式執行完畢 " + "-" * 20)
    print("🎉 全部任務完成！")
    if enable_google_drive_integration:
        print(f"ℹ️ 請檢查您的 Google Drive/{project_folder} 資料夾以確認產出檔案。")
    else:
        print(f"ℹ️ 本地測試模式執行完畢。產出檔案位於 Colab 環境的 /content/{project_folder}，關閉分頁後將清除。")
except SystemExit as e: # 捕捉 sys.exit()，正常結束
    print("\nℹ️ 執行已中止。這可能是您主動停止，或是在前置檢查階段發現問題而自動停止，屬於正常情況。")
except Exception as e:
    print("\n❌ 哎呀！執行過程中似乎遇到了一個預期外的問題。")
    print(f"\n  【技術性錯誤細節】: {e}")
    print("\n👉 您可以怎麼做？")
    log_dir_suggestion = os.path.join(project_folder, "99_logs") if 'project_folder' in locals() else "專案資料夾內的 99_logs"
    print(f"   1. 您可以先去專案日誌檔中尋找更詳細的線索，它通常位於您的 {log_dir_suggestion} 目錄中。")
    print("   2. 最常見的解決方法是：點擊上方選單的「執行階段」->「中斷並刪除執行階段」，然後從頭開始重新執行一次。")
